## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
#esercizio 1

file_name="data_000637.txt"
data_tot=pd.read_csv(file_name)
data = pd.DataFrame(data_tot[0:10000], index=np.arange(10000), columns=["HEAD","FPGA","TDC_CHANNEL","ORBIT_CNT","BX_COUNTER","TDC_MEAS"])
MAX = data.groupby("ORBIT_CNT").max()
MIN = data.groupby("ORBIT_CNT").min()
delta = MAX -MIN
x = pd.Series.mean(delta["BX_COUNTER"])


In [ ]:
#esercizio 2

time = data["TDC_MEAS"].sum()*25





In [ ]:
#esercizio 3

absolute = data["ORBIT_CNT"].tail(1)*25*x + data["BX_COUNTER"].tail(1)*25
print(absolute) #in nanosecondi


In [ ]:
#esercizo 5

data["HEAD"] = pd.Series(np.random.randint(0,2,len(data["HEAD"])), index=data.index)


In [ ]:
#esercizio 6

lista_drop = [data.index[i] for i in range(0,10000) if data["HEAD"][i] == 0]

In [ ]:
#esercizio 7


lista_drop1 = [data.index[i] for i in range(0,10000) if data["FPGA"][i] == 0]
lista_drop2 = [data.index[i] for i in range(0,10000) if data["FPGA"][i] == 1]

fpga1 = data.drop(lista_drop1)
fpga2 = data.drop(lista_drop2)


primoisto = fpga1["TDC_CHANNEL"].plot.hist(  bins=140)


In [ ]:
secondoisto = fpga2["TDC_CHANNEL"].plot.hist(  bins=140)

In [ ]:
#esercizio 8

noisy1 = fpga1.groupby("TDC_CHANNEL").sum()
noisy1

noisy2 = fpga2.groupby("TDC_CHANNEL").sum()
noisy2


In [ ]:
#esercizio 9

conta_orbite1 = fpga1.groupby("ORBIT_CNT").sum()


conta_orbite2 = fpga2.groupby("ORBIT_CNT").sum()
conta_orbite2

#ho messo la funzone sum a caso, ciò che mi interessa è il numero di righe
